<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/08_document_classification_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNNを使った文書分類
* RNNの出力を文書の潜在表現として利用し、文書分類を行う。
 * トークンの埋め込みも含めて学習する。

## 全体の準備
* ランタイムのタイプをGPUにしておく。


* `torchdata`をインストールしておく。

In [ ]:
!pip install torchdata

* 再現性の確保については下記を参照。
 * https://pytorch.org/docs/stable/notes/randomness.html

In [ ]:
import random
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext import datasets

SEED = 123

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## IMDbデータの準備
* ここでIMDbデータセットの読み込みにつかう`torchtext.datasets`については、下記を参照。
 * https://pytorch.org/text/stable/datasets.html
 * https://torchtext.readthedocs.io/en/latest/datasets.html

### 訓練データから語彙集合を作成
* データの読み込みや前処理は`torchtext`の旧バージョン（0.9より前）とかなり変わっている。
 * なので、Web検索で`torchtext`について情報収集するときは要注意。

In [ ]:
import torchtext
torchtext.__version__

* データセットの訓練データ部分をもとに語彙セットを作成
 * 語彙セットを作成するとき、テストデータ部分を使わないように。

* 今回の特殊なトークンの設定
 * 未知語は`<unk>`という特殊なトークンで置き換える。
 * 同じミニバッチ内で、トークン列の長さが揃っていないとき、末尾は`<pad>`という特殊なトークンで埋める。

In [ ]:
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
#tokenizer = get_tokenizer('spacy') # やや時間がかかる。

# IMDbデータの訓練データ部分のイテレータを取得
train_iter = IMDB(split='train')

# 各文書をtokenizeするヘルパ関数
def yield_tokens(data_iter):
  for _, text in data_iter:
    yield tokenizer(text)

# 最小の出現頻度をmin_freqで指定して語彙サイズを調整する
vocab = build_vocab_from_iterator(
    yield_tokens(train_iter),
    specials=["<unk>", "<pad>"],
    min_freq=5,
    )

# OoVトークンには"<unk>"のインデックスを返すことにする
vocab.set_default_index(vocab["<unk>"]) 

In [ ]:
print(len(vocab))

* 以下のように、トークン列をインデックス列へ変換できる。

In [ ]:
vocab(['here', 'is', 'an', 'example'])

* 特殊なトークンには、以下のインデックスが割り振られている。

In [ ]:
vocab(['<unk>', '<pad>'])

* 単語とインデックスの間の変換は、以下のように行うことができる。

In [ ]:
vocab.get_stoi()['apple']

In [ ]:
vocab.get_itos()[7316]

### IMDBデータを分割してデータセットを作成
* 訓練データとテストデータ、二つのイテレータを、データセットへ変換する。
 * こうしないとsplitできない。
 * テストデータの方も、こうしないと評価のときエラーが出てしまう。

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

train_iter, test_iter = IMDB()

train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

* 訓練データ部分から検証データを切り出す

In [ ]:
num_train = int(len(train_dataset) * 0.9)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])
print(f"train size={len(split_train_)} valid size={len(split_valid_)}")

### データセットからデータローダを作成

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 8
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True)

* 上のデータローダは、そのままRNNの入力としては使えないミニバッチを与える。
 * 生のテキストをそのままRNNに与えるわけにはいかない。

In [ ]:
next(iter(train_dataloader))

* RNNの入力に使える形式のミニバッチを作るための関数を定義する。
 * テキストをtokenizeし、インデックス列へ変換し、paddingして長さを揃える関数。

In [ ]:
from torch.nn.utils.rnn import pad_sequence

# paddingに使うトークンのインデックスを取得
PAD_IDX = vocab.get_stoi()['<pad>']

def collate_batch(batch):
  labels = {'neg':0, 'pos':1} #ラベルは0/1に変換
  label_list, text_list = [], []
  for _label, _text in batch:
    label_list.append(labels[_label])
    processed_text = torch.tensor(vocab(tokenizer(_text)), dtype=torch.int64)
    text_list.append(processed_text)
  label_list = torch.tensor(label_list, dtype=torch.int64)
  text_list = pad_sequence(text_list, batch_first=False, padding_value=PAD_IDX)
  return label_list.to(device), text_list.to(device)

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64
train_dataloader = DataLoader(
    split_train_, 
    batch_size=BATCH_SIZE, 
    shuffle=True,
    collate_fn=collate_batch,
    )

In [ ]:
mini_batch = next(iter(train_dataloader))

In [ ]:
mini_batch

In [ ]:
mini_batch[1].shape

In [ ]:
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch)

## モデルの定義
* LSTMを使う（GRUに変えても良い）
 * http://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [ ]:
INPUT_DIM = len(vocab)
NUM_CLASS = 2
EMBED_DIM = 64
HIDDEN_DIM = 64

In [ ]:
class RNNTextSentiment(nn.Module):
  def __init__(self, emb_dim, hid_dim,
               num_class, vocab_size, padding_idx, p=0.0):
    super().__init__()

    self.input_dim = vocab_size
    self.emb_dim = emb_dim
    self.hid_dim = hid_dim
    self.dropout = p

    self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=padding_idx)
    self.rnn = nn.LSTM(emb_dim, hid_dim)
    self.fc = nn.Linear(hid_dim * 2, num_class)
    self.dropout = nn.Dropout(p=p)

  def forward(self, src):
    # srcの形は[単語列長, バッチサイズ]

    embedded = self.dropout(self.embedding(src))
    # embeddedの形は[単語列長, バッチサイズ, 埋め込み次元数]

    outputs, (hidden, _) = self.rnn(embedded)
    # outputsの形は[単語列長, バッチサイズ, 隠れ状態の次元数]
    # hiddenの形は[1, バッチサイズ, 隠れ状態の次元数]

    mean_outputs = outputs.mean(0)
    hidden = hidden.squeeze()
    # mean_outputsの形は[バッチサイズ, 隠れ状態の次元数]
    # hiddenの形は[バッチサイズ, 隠れ状態の次元数]
    output = self.fc(torch.cat((mean_outputs, hidden), dim=1))

    return output

* モデルのインスタンスを得る

In [ ]:
model = RNNTextSentiment(
    EMBED_DIM, HIDDEN_DIM, NUM_CLASS, INPUT_DIM,
    padding_idx=PAD_IDX, p=0.5,
    ).to(device)

## 最適化アルゴリズムの設定

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

パラメータの数を数えてみる。

In [ ]:
def count_parameters(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

## 損失関数の設定

In [ ]:
criterion = nn.CrossEntropyLoss()

## 訓練用の関数

In [ ]:
def train(model, loader, optimizer, criterion, clip):
  model.train()

  epoch_loss = 0.
  epoch_acc = 0.
  for label, text in loader:
    output = model(text)
    loss = criterion(output, label)
    loss.backward()

    nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizer.step()
    optimizer.zero_grad()

    epoch_loss += loss.item()
    epoch_acc += (output.argmax(1) == label).sum().item()

  return epoch_loss / len(loader), epoch_acc / len(loader.dataset)

## 評価用の関数

In [ ]:
def evaluate(model, loader, criterion):
  model.eval()

  epoch_loss = 0.
  epoch_acc = 0.
  with torch.no_grad():
    for label, text in loader:
      output = model(text)
      loss = criterion(output, label)
      epoch_loss += loss.item()
      epoch_acc += (output.argmax(1) == label).sum().item()

  return epoch_loss / len(loader), epoch_acc / len(loader.dataset)

* 時間表示用の関数

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time // 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

## 学習の実行

In [ ]:
N_EPOCHS = 10
CLIP = 1.

for epoch in range(1, N_EPOCHS + 1):

  start_time = time.time()
  train_loss, train_acc = train(model, train_dataloader, optimizer, criterion, CLIP)
  valid_loss, valid_acc = evaluate(model, valid_dataloader, criterion)
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)

  print(f'Epoch {epoch} | time in {epoch_mins} minutes, {epoch_secs} seconds')
  print(f'\tLoss {train_loss:.4f} (train)\t|\tAcc {train_acc * 100:.1f}% (train)')
  print(f'\tLoss {valid_loss:.4f} (valid)\t|\tAcc {valid_acc * 100:.1f}% (valid)')

# 課題
* 上のコードを動かして、感情分析を実践してみよう。
 * テストデータで評価してみる。
* 余裕があれば、ハイパーパラメータをチューニングして、分類性能を上げてみよう。

## テストデータで評価

In [ ]:
loss, acc = evaluate(model, test_dataloader, criterion)
print(f'\tLoss {loss:.4f} (test)\t|\tAcc {acc * 100:.1f}% (test)')